In [1]:
import pandas as pd
import numpy as np
# import scipy.sparse as sp
import psycopg2
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
conn = psycopg2.connect(
    host="my-db.c8684qazpg5m.us-east-1.rds.amazonaws.com",
    database="movie_recommendation_db",
    user="hemanth",
    password="hemanth123",
    port=5432
)
cur = conn.cursor()

In [3]:
# Fetch the ratings and movies data using SQL queries
ratings_query = "SELECT userid, movieid, rating FROM ratings"
movies_query = "SELECT movieid, title FROM movies"
ratings = pd.read_sql(ratings_query, conn)
movies = pd.read_sql(movies_query, conn)

<ipython-input-3-6f2e08276ed7>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql(ratings_query, conn)
<ipython-input-3-6f2e08276ed7>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies = pd.read_sql(movies_query, conn)


In [4]:
data = pd.merge(ratings, movies, on="movieid")

In [5]:
data = data.pivot_table(index='userid', columns='title', values='rating')
data = data.fillna(0)

In [6]:
data

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


In [7]:
data_tensor = torch.FloatTensor(data.values)

In [8]:
# Define model parameters
input_size = data_tensor.shape[1]
hidden_size_1 = 512
hidden_size_2 = 256
output_size = input_size

In [9]:
# Define model class
class StackedAutoencoder(nn.Module):
    def __init__(self):
        super(StackedAutoencoder, self).__init__()
        # Encoder layers
        self.linear_1 = nn.Linear(input_size, hidden_size_1)
        self.linear_2 = nn.Linear(hidden_size_1, hidden_size_2)
        # Decoder layers
        self.linear_3 = nn.Linear(hidden_size_2, hidden_size_1)
        self.linear_4 = nn.Linear(hidden_size_1, output_size)
        # Activation function
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Encode input
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.relu(x)
        # Decode input
        x = self.linear_3(x)
        x = self.relu(x)
        x = self.linear_4(x)
        return x

In [10]:
# Create model instance
model = StackedAutoencoder()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Define training parameters
epochs = 30
batch_size = 32
train_size = int(0.8 * len(data_tensor))
test_size = len(data_tensor) - train_size
train_data, test_data = torch.utils.data.random_split(data_tensor, [train_size, test_size])

In [11]:
# Train the model
for epoch in range(epochs):
    # Shuffle the data
    perm = torch.randperm(data_tensor.size(0))
    data_tensor = data_tensor[perm]
    # Loop over batches
    for i in range(0, data_tensor.size(0), batch_size):
        # Get batch data
        batch_data = data_tensor[i:i+batch_size]
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        output = model(batch_data)
        # Compute loss
        loss = criterion(output, batch_data)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    # Print loss every epoch
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
    # print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 10, Loss: 0.0341
Epoch 20, Loss: 0.0640
Epoch 30, Loss: 0.1655


In [12]:
# Evaluate the model on the testing set
test_loss = 0.0
with torch.no_grad():
    for i in range(0, test_data.dataset.size(0), batch_size):
        # Get batch data
        batch_data = test_data.dataset[i:i+batch_size]
        # Forward pass
        output = model(batch_data)
        # Compute loss
        loss = criterion(output, batch_data)
        test_loss += loss.item()
test_loss /= len(test_data)
print(f'Test Loss: {test_loss:.4f}')

Test Loss: 0.0402


In [13]:
def get_recommendations(user_id, k):
    # Get the user ratings vector
    user_ratings = data_tensor[user_id]
    # Forward pass through the model
    output = model(user_ratings)
    # Get the predicted ratings
    pred_ratings = output.detach().numpy()
    # Get the movie titles
    movie_titles = data.columns
    # Sort the movies by predicted ratings in descending order
    sorted_indices = np.argsort(pred_ratings)[::-1]
    sorted_movies = movie_titles[sorted_indices]
    sorted_ratings = pred_ratings[sorted_indices]
    # Return the top k movies and ratings
    return sorted_movies[:k], sorted_ratings[:k]

In [14]:
movies, ratings = get_recommendations(5, 10)

In [15]:
print(movies)

Index(['Forrest Gump (1994)', 'Shawshank Redemption, The (1994)',
       'Braveheart (1995)', 'Fugitive, The (1993)', 'Pulp Fiction (1994)',
       'Apollo 13 (1995)', 'Jurassic Park (1993)', 'Dances with Wolves (1990)',
       'True Lies (1994)', 'Silence of the Lambs, The (1991)'],
      dtype='object', name='title')


In [16]:
ratings

array([2.738927 , 2.618606 , 2.4775028, 2.4228482, 2.3882961, 2.340551 ,
       2.2162457, 2.1702175, 2.0763376, 2.0747294], dtype=float32)

In [17]:
data_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])